In [0]:
spark

In [0]:

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Feale","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"
emp = spark.createDataFrame(emp_data, emp_schema)
emp.limit(2).show()

In [0]:
from pyspark.sql.functions import *
emp.withColumn("Newgender",when(col("gender")=='Male','M').when(col("gender")=='Female','F').otherwise(col("gender"))).show()
emp.withColumn("Newgender",expr("case when gender='Male' then 'M' when gender='Female' then 'F' else gender end")).show()

In [0]:
emp.select('*',to_date(col('hire_date'),'yyyy-MM-dd').alias('hire_date_new')).show()

In [0]:
emp.select("*",current_timestamp().alias('current_timestamp')).show()

In [0]:
emp.selectExpr("*","case when gender='Male' then 'M' when gender='Female' then 'F' else null end as Newgender").show()

In [0]:
emp.select("*",date_format(col('hire_date'),'dd-MM-yyyy').alias('hire_date_new')).show()
emp.select("*")

In [0]:
emp.select('*',date_format(current_timestamp(),'yyyy').alias('Year')).show()

In [0]:
emp.orderBy(col('salary').desc()).show()

In [0]:
emp.groupBy(col('department_id')).agg(sum('salary').alias('Deptsalary'),count('employee_id').alias('empcnt')).filter(col('empcnt')>2).show()


In [0]:
#max salary by deparment
from pyspark.sql.window import Window
ws = Window.partitionBy(col('department_id')).orderBy(col('salary').desc())
emp.withColumn('MaxSalary',max('salary').over(ws)).show()

In [0]:
#find the second Highest Salary
ws = Window.partitionBy(col('department_id')).orderBy(col('salary').desc())
emp.withColumn('rownum',row_number().over(ws)).filter(col('rownum')==2).show()

In [0]:
emp.selectExpr("*","row_number() over(partition by department_id order by salary desc) as rownum").filter(col('rownum')==2).show()

In [0]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]
emp = spark.createDataFrame(emp_data,emp_schema)


dept_schema = "department_id string, department_name string, city string, country string, budget string"
dept = spark.createDataFrame(dept_data,dept_schema)

In [0]:
emp.rdd.getNumpartitions()
dept.rdd.getNumPartitions()
emp.show()
dept.show()

In [0]:
empapr= emp.repartition(4,'department_id')

In [0]:
from pyspark.sql.functions import spark_partition_id
empapr.withColumn('parttionID',spark_partition_id()).show()


In [0]:
spark